# 2. Multi-instrument one-dimensional analysis

In the last notebook, we have extracted counts vs energy, but this is a physical quantity (or information) filtered through our detector system. What we really aim at estimating is the gamma-ray flux of the source, in particular, the flux over a large band in energy (spectrum).

In [ ]:
# - basic dependencies
import numpy as np
import astropy.units as u
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# - Gammapy dependencies
from gammapy.datasets import SpectrumDatasetOnOff, Datasets
from gammapy.modeling import Parameter
from gammapy.modeling.models import (
    SpectralModel,
    PowerLawSpectralModel,
    LogParabolaSpectralModel,
    SkyModel,
    create_crab_spectral_model,
)
from gammapy.modeling import Fit
from gammapy.estimators import FluxPointsEstimator

In [ ]:
# read image
img = mpimg.imread("figures/crab_sed_hess.png")
# display image
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(img)
ax.axis("off")
plt.show()

In [ ]:
# read image
img = mpimg.imread("figures/crab_sed_magic.png")
# display image
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(img)
ax.axis("off")
plt.show()

To obtain a measurement of the specturm, it is commonly assumed that a simple analytical function describes the gamma-ray (differential) flux vs energy. The most common example is a power-law:

$$
\frac{{\rm d}\phi}{{\rm d}E}(E; \Phi_0, \Gamma, E_0)\,[{\rm TeV}^{-1}\,{\rm cm}^{-2}\,{\rm s}^{-1}] = 
\Phi_0 \left(\frac{E}{E_0}\right)^{-\Gamma}.
$$

Note that we have a _continuous_ dependency on the energy (we want to measure the flux as a function of the energy) and we have a _parametric_ dependency on few parameters that describe our analytical spectrum, in this case the normalisation $\Phi_0$, the reference energy, $E_0$, and the spectral index, $\Gamma$. From now on, we will represent all the model parameters with an array $\hat{\theta}$.

Let us now plot some of the examples of analytical spectra available in `Gammapy`:

In [ ]:
energy_bounds = [100 * u.GeV, 10 * u.TeV]
pwl = PowerLawSpectralModel()
pwl.plot(energy_bounds, label="power law")
lp = LogParabolaSpectralModel(
    amplitude=7e-11 * u.Unit("TeV-1 cm-2 s-1"), reference=100 * u.GeV
)
lp.plot(energy_bounds, label="log parabola")
plt.legend()
plt.show()

How can we adjust the parameters $\hat{\theta}$ of the analytical spectrum to the counts that we have extracted? That's where the IRF come into place. The IRF allow us to go from gamma-ray flux to instrument counts and vice versa. By folding (i.e. convolving) the analytical flux model with the response of the system we can obtain counts vs energy. We basically transform the absolute flux quantity relative to the detector. The number of model (or _predicted_) counts in a given energy bin $\Delta E'$ is:

$$
g_{i}(\hat{\theta}) = 
    t_{\rm eff} \int_{\Delta E'} {\rm d}E' \int_{0}^{\infty} {\rm d}E \;
    A_{\rm eff}(E) M(E'|E) \frac{{\rm d}\phi}{{\rm d}E}(E; \hat{\theta}),
$$

where $A_{\rm eff}(E)$ represent the effective area and $M(E'|E)$ the energy dispersion, i.e. the PDF of the energy estimator.

How we decide which (predicted) counts better describe our model? With a likelihood, we assume that the counts in each bin are described by Poissonian statistics. The total likelihood for our dataset, accounting for the observed and estimated counts in each energy bin, reads:

$$ 
\mathcal{L}(\hat{\theta}) = 
    \Pi_{i=1}^{N} {\rm Pois}(g_i(\hat{\theta}) + b_i; N_{{\rm on}, i}) {\rm Pois}(b_i / \alpha; N_{{\rm off}, i})
$$

where $g_i$ is the number of predicted counts, $\alpha$ the ratio between the on and off region exposures. $b_i$, the number of background counts, is typically treated as a nuisance parameter. The likelihood is maximised by varying the spectral parameters $\hat{\theta}$, and hence the number of predicted counts.

## 2.1. Predicting the counts for a given data set
Let us load the data sets we created in the previous notebook. We select also an energy range compatible with the instruments performance and with the range of events observed.

In [ ]:
# H.E.S.S. data sets
obs_ids_hess = [23523, 23526, 23559, 23592]
# declare the energy range to be used for the fitting
e_min_hess = 0.66 * u.TeV
e_max_hess = 30 * u.TeV

# load H.E.S.S. spectral datasets
datasets_hess = Datasets()
for obs_id in obs_ids_hess:
    dataset = SpectrumDatasetOnOff.read(f"results/spectra/hess/pha_obs_{obs_id}.fits")
    # set the proper energy ranges
    dataset.mask_fit = dataset.counts.geom.energy_mask(e_min_hess, e_max_hess)
    datasets_hess.append(dataset)


# MAGIC data sets
obs_ids_magic = [5029747, 5029748]
# declare the energy range to be used for the fitting
e_min_magic = 0.08 * u.TeV
e_max_magic = 10 * u.TeV

# load MAGIC spectral datasets
datasets_magic = Datasets()
for obs_id in obs_ids_magic:
    dataset = SpectrumDatasetOnOff.read(f"results/spectra/magic/pha_obs_{obs_id}.fits")
    # set the proper energy ranges
    dataset.mask_fit = dataset.counts.geom.energy_mask(e_min_magic, e_max_magic)
    datasets_magic.append(dataset)


# LST data sets
obs_ids_lst = [7253, 7254, 7255, 7256, 7274, 7275, 7276, 7277]
# declare the energy range to be used for the fitting
e_min_lst = 0.06 * u.TeV
e_max_lst = 30 * u.TeV

# load LST spectral datasets
datasets_lst = Datasets()
for obs_id in obs_ids_lst:
    dataset = SpectrumDatasetOnOff.read(f"results/spectra/lst/pha_obs_{obs_id}.fits")
    # set the proper energy ranges
    dataset.mask_fit = dataset.counts.geom.energy_mask(e_min_lst, e_max_lst)
    datasets_lst.append(dataset)

Once we have the data sets we can specify a model for them, `Gammapy` will automatically convolve it with the response of the system and compute (predict) the counts. Here an example.

In [ ]:
# define the model to be fitted
spectral_model = LogParabolaSpectralModel(
    amplitude=5e-12 * u.Unit("TeV-1 cm-2 s-1"),
    reference=1 * u.TeV,
    alpha=2.3 * u.Unit(""),
    beta=0.1 * u.Unit(""),
)
# trick to facilitate ULs computation
spectral_model.amplitude.min = 1e-20
spectral_model.amplitude.max = 1e-5
print(spectral_model)

model = SkyModel(spectral_model=spectral_model, name="CrabNebula")

datasets_lst.models = [model]

In [ ]:
datasets_lst[0].plot_excess()
plt.show()

We can also check the contribution of this data set to the total likelihood.

In [ ]:
datasets_lst[0].stat_sum()

As we can see, the counts predicted by the model we specified for the data set are one order of magnitude below the observed source counts. 

To get a feeling of how the fit work, let us go back to the definition of the spectral analytical function and change the amplitude parameter to `5e-11 * u.Unit("TeV-1 cm-2 s-1")`. What happens to the predicted counts? And what happens to the likelihood statistics?

In [ ]:
datasets_lst.models[0].spectral_model.amplitude.value

In [ ]:
datasets_lst.models[0].spectral_model.amplitude.value = 5e-11

datasets_lst[0].plot_excess()
plt.show()

In [ ]:
datasets_lst[0].stat_sum()


You have now an intuition of how the fitting routine work. The parameters will be changed until the best match between observed and predicted counts is observed. This agreement is statistically quantified by the likelihood, which is what is actually maximised.

## 2.2. Spectrum of the Crab Nebula as seen by LST-1
The likelihood maximisation is performed by the `gammapy.Fit` class using the `gammapy.Dataset` and the model we have asigned to it. 

In [ ]:
# run the fit!
fit = Fit()
results = fit.run(datasets=datasets_lst)
print(results)
print(spectral_model)

We can check, after the optimisation, how the predicted counts adapt to the observed one

In [ ]:
ax_spectrum, ax_residuals = datasets_lst[0].plot_fit()
plt.show()

and check also the value of the likelihood per each of the data sets.

In [ ]:
datasets_lst[0].stat_sum()

We can now plot the spectrum obtained by our fitting procedure. It is common in high-energy astrophysics, to represent instead of the differential flux the Spectral Energy Distribution (SED) obtained as $E^2\frac{{\rm d}\phi}{{\rm d}E} [{\rm TeV}^{-1}\,{\rm cm}^{-2}\,{\rm s}^{-1}]$ that represent the how the emitted power is distributed over the electromagnetic spectrum. We compare the results we have obtained with a theoretical model of the SED and with another measurement performed by MAGIC.

In [ ]:
fig, ax = plt.subplots()

plot_kwargs = {
    "sed_type": "e2dnde",
    "yunits": u.Unit("TeV cm-2 s-1"),
    "xunits": u.GeV,
}

energy_range = [e_min_lst, e_max_lst]

spectral_model.plot(
    energy_range,
    ax=ax,
    color="crimson",
    ls="-",
    label="LST",
    **plot_kwargs,
)
spectral_model.plot_error(
    energy_range, ax=ax, facecolor="crimson", alpha=0.4, **plot_kwargs
)

crab_meyer = create_crab_spectral_model("meyer")
crab_meyer.plot(
    energy_range,
    ax=ax,
    label="Meyer et al. (2010)",
    color="k",
    ls="--",
    **plot_kwargs,
)

crab_magic = create_crab_spectral_model("magic_lp")
crab_magic.plot(
    energy_range,
    ax=ax,
    label="MAGIC Collaboration (2015)",
    color="dodgerblue",
    ls="--",
    **plot_kwargs,
)

ax.legend()

ax.set_xlabel(r"$E\,/\,{\rm GeV}$")
ax.set_ylabel(
    r"$E^2 {\rm d}\phi/{\rm d}E\,/\,({\rm TeV}\,{\rm cm}^{-2}\,{\rm s}^{-1})$"
)
ax.set_ylim([8e-13, 2e-10])
plt.show()

Finally we also compute spectral data points or _flux points_. They are basically a flux measurement over different energy bins. Broadly speaking, they have two objectives:
1) when we fit the broad-band spectrum via the likelihood maximisation - as we have just shown - we are assuming that the entire spectrum is described by a single analytic smooth functions. This type of analysis is therefore not sensitive to any sharp feature in the spectrum;
2) we are adopting an analytical function to provide a broad-band flux measurements, yet, at a later stage, other astrophysicists might want to fit a physical model to our gamma-ray data. In order to do so they should have access to the data sets (counts + IRF) that we are using. In case this data are proprietary we can provide a simplified and easier to handle flux measurements.

To compute flux points, Gammapy starts from the best-fit spectrum obtained from the broad-band likelihood fit and re-adjusts it spearately to the events within each energy bin. Note theat only the amplitude of the spectrum is re-fitted. Basically, the spectrum we obtained from the broad-band likelihood fit is scaled up and down to adjust independently to the counts in each energy bin.

Let us use the same binning we have used in estimated energies and select an interval compatible with what used in the likelihood fit.

In [ ]:
datasets_lst[0].counts.geom.axes["energy"].edges[4:-2]

In [ ]:
energy_edges = datasets_lst[0].counts.geom.axes["energy"].edges[4:-2]
fpe_lst = FluxPointsEstimator(
    energy_edges=energy_edges, source="CrabNebula", selection_optional="all"
)
flux_points_lst = fpe_lst.run(datasets=datasets_lst)

In [ ]:
fig, ax = plt.subplots()
flux_points_lst.plot(ax=ax, sed_type="e2dnde", color="darkorange", label="flux points")
flux_points_lst.plot_ts_profiles(ax=ax, sed_type="e2dnde")
spectral_model.plot(
    energy_range,
    ax=ax,
    color="crimson",
    ls="--",
    lw=2,
    sed_type="e2dnde",
    label="broad-band likelihood fit",
)
ax.set_xlim([40, 2.5e4])
ax.set_xlabel(r"$E\,/\,{\rm GeV}$")
ax.set_ylabel(
    r"$E^2 {\rm d}\phi/{\rm d}E\,/\,({\rm TeV}\,{\rm cm}^{-2}\,{\rm s}^{-1})$"
)
ax.legend()
plt.show()

## 2.3. Exercises for this tutorial

### Exercise 2.3.1.
Obtain a multi-instrument measurement of the Crab Nebula spectrum by fitting the datasets from all the instruments at once. As LST-1 data contain much more staitstics than the others, use only the first run of that data set. Compute the flux points for each instrument and overplot them on the final spectrum.

In [ ]:
# %load ./solutions/solution_notebook_2.py